In [1]:
#pip install pandas numpy scikit-learn xgboost joblib streamlit python-dotenv pymongo tqdm torch transformers


In [2]:
# # preprocess_and_augment.py
# #tf-idf
# import pandas as pd
# import numpy as np
# import ast
# import joblib
# import os
# import json
# from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
# from tqdm import tqdm

# # --- CONFIGURATION ---
# TRAIN_PATH = 'dataset/22687585 (1)/release_train_patients/release_train_patients'
# SYMPTOM_MAP_PATH = 'dataset/22687585 (1)/release_evidences.json'
# OUTPUT_DIR = 'model_outputs/'
# AUGMENTED_DATA_PATH = 'augmented_training_data.csv'
# os.makedirs(OUTPUT_DIR, exist_ok=True)
# CHUNKSIZE = 5000

# # --- DATA AUGMENTATION FUNCTION ---
# def augment_data(df: pd.DataFrame, feature_columns: list) -> pd.DataFrame:
#     augmented_rows = []
#     for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Augmenting chunk"):
#         # Dropout
#         dropout_row = row.copy()
#         present_features = [col for col in feature_columns if row[col] == 1]
#         if len(present_features) > 1:
#             feature_to_drop = np.random.choice(present_features)
#             dropout_row[feature_to_drop] = 0
#             augmented_rows.append(dropout_row)
#         # Noise
#         noise_row = row.copy()
#         absent_features = [col for col in feature_columns if row[col] == 0]
#         if len(absent_features) > 0:
#             feature_to_add = np.random.choice(absent_features)
#             noise_row[feature_to_add] = 1
#             augmented_rows.append(noise_row)
#     return pd.concat([df, pd.DataFrame(augmented_rows)], ignore_index=True)

# # --- MAIN PREPROCESSING SCRIPT ---
# if __name__ == "__main__":
#     # === PASS 1: FIT ENCODERS & GATHER STATS ===
#     print("--- Pass 1: Fitting encoders and gathering stats ---")
#     with open(SYMPTOM_MAP_PATH, 'r', encoding='utf-8') as f:
#         symptom_map_json = json.load(f)
#     symptom_map = {k: v['name'].replace('_', ' ') for k, v in symptom_map_json.items() if isinstance(v, dict) and 'name' in v}

#     all_symptoms = set()
#     age_series = pd.Series(dtype=float)
#     for chunk in pd.read_csv(TRAIN_PATH, chunksize=CHUNKSIZE):
#         evidences = chunk['EVIDENCES'].apply(ast.literal_eval)
#         for symptom_list in evidences:
#             all_symptoms.update(symptom_list)
#         age_series = pd.concat([age_series, chunk['AGE']])
#     global_age_median = age_series.median()

#     mlb = MultiLabelBinarizer(classes=sorted(list(all_symptoms)))
#     mlb.fit([[]]) # Fit on empty to finalize classes
#     le = LabelEncoder().fit(['Low Risk', 'Moderate Risk', 'High Risk'])
#     print("✅ Encoders fitted successfully.")

#     # === PASS 2: PROCESS AND AUGMENT DATA IN CHUNKS ===
#     print("\n--- Pass 2: Processing and augmenting data in chunks ---")
#     if os.path.exists(AUGMENTED_DATA_PATH):
#         os.remove(AUGMENTED_DATA_PATH)
#     header_written = False

#     for i, chunk in enumerate(pd.read_csv(TRAIN_PATH, chunksize=CHUNKSIZE)):
#         print(f"  - Processing chunk {i+1}...")
#         # Handle missing values
#         chunk['AGE'] = chunk['AGE'].fillna(global_age_median)
#         chunk['EVIDENCES_LIST'] = chunk['EVIDENCES'].apply(ast.literal_eval)

#         # Transform symptoms
#         symptom_features = mlb.transform(chunk['EVIDENCES_LIST'])
#         df_symptoms = pd.DataFrame(symptom_features, columns=mlb.classes_, index=chunk.index)

#         # Create risk levels
#         high_risk = ['Myocardial infarction', 'Pneumonia', 'Appendicitis', 'Pulmonary embolism']
#         moderate_risk = ['Malaria', 'Dengue', 'Tuberculosis', 'Gastroenteritis']
#         chunk['risk_level'] = chunk['PATHOLOGY'].apply(
#             lambda d: 'High Risk' if d in high_risk
#             else 'Moderate Risk' if d in moderate_risk
#             else 'Low Risk'
#         )

#         # Encode sex
#         sex_encoded = pd.get_dummies(chunk['SEX'], drop_first=True, prefix='SEX')
#         if 'SEX_M' not in sex_encoded.columns:
#             sex_encoded['SEX_M'] = 0

#         # Combine all features
#         df_chunk_processed = pd.concat([
#             chunk[['AGE', 'risk_level']],
#             sex_encoded[['SEX_M']],
#             df_symptoms
#         ], axis=1).fillna(0)

#         # Augment data
#         feature_cols = mlb.classes_.tolist()
#         df_chunk_augmented = augment_data(df_chunk_processed, feature_cols)

#         # Save to CSV
#         df_chunk_augmented.to_csv(
#             AUGMENTED_DATA_PATH,
#             mode='a',
#             header=not header_written,
#             index=False
#         )
#         header_written = True

#     print(f"\n✅ Final augmented dataset saved to '{AUGMENTED_DATA_PATH}'")

#     # === 3. SAVE THE FITTED ARTIFACTS ===
#     print("\nSaving all encoders and artifacts...")
#     joblib.dump(le, os.path.join(OUTPUT_DIR, 'risk_label_encoder.pkl'))
#     joblib.dump(mlb, os.path.join(OUTPUT_DIR, 'symptom_encoder.pkl'))
#     joblib.dump(symptom_map, os.path.join(OUTPUT_DIR, 'symptom_map.pkl'))
#     joblib.dump(
#         df_chunk_augmented.columns.drop('risk_level').tolist(),
#         os.path.join(OUTPUT_DIR, 'final_feature_columns.pkl')
#     )
#     print(f"✅ All artifacts saved to '{OUTPUT_DIR}' directory.")

In [3]:
# # train.py
# #tf-idf
# import pandas as pd
# import numpy as np
# import ast
# import joblib
# import os
# import json
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
# from sklearn.metrics import classification_report
# import xgboost as xgb
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD
# from sklearn.utils import class_weight
# from tqdm import tqdm

# # =======================================
# # CONFIGURATION
# # =======================================
# TRAIN_PATH = '/content/drive/MyDrive/Medaid/release_train_patients'
# SYMPTOM_MAP_PATH = '/content/drive/MyDrive/Medaid/release_evidences.json'
# OUTPUT_DIR = 'model_outputs/'
# os.makedirs(OUTPUT_DIR, exist_ok=True)

# # =======================================
# # DATA AUGMENTATION FUNCTION
# # =======================================
# def augment_data(df: pd.DataFrame, symptom_columns: list) -> pd.DataFrame:
#     """Augments data with dropout and noise for better generalization."""
#     print(f"\n[7/9] Augmenting data... Original size: {len(df)} rows.")
#     augmented_rows = []
#     for _, row in tqdm(df.iterrows(), total=df.shape[0], desc="Augmenting data"):
#         # Dropout
#         dropout_row = row.copy()
#         present_features = [col for col in symptom_columns if row[col] == 1]
#         if len(present_features) > 1:
#             feature_to_drop = np.random.choice(present_features)
#             dropout_row[feature_to_drop] = 0
#             augmented_rows.append(dropout_row)
#         # Noise
#         noise_row = row.copy()
#         absent_features = [col for col in symptom_columns if row[col] == 0]
#         if absent_features:
#             feature_to_add = np.random.choice(absent_features)
#             noise_row[feature_to_add] = 1
#             augmented_rows.append(noise_row)
#     augmented_df = pd.concat([df, pd.DataFrame(augmented_rows)], ignore_index=True)
#     print(f"✅ Augmentation complete. New size: {len(augmented_df)} rows.")
#     return augmented_df

# # =======================================
# # MAIN TRAINING SCRIPT
# # =======================================
# if __name__ == "__main__":
#     # --- 1. Load Data & Symptom Map ---
#     print("\n[1/9] Loading data and symptom map...")
#     df_full = pd.read_csv(TRAIN_PATH) # Load the full dataframe first

#     # --- NEW STEP: DOWNSAMPLE THE DATA ---
#     N_SAMPLES_TO_USE = 100000 # You can adjust this number
#     if len(df_full) > N_SAMPLES_TO_USE:
#         print(f"\nDownsampling data from {len(df_full)} to {N_SAMPLES_TO_USE} rows...")
#         df = df_full.sample(n=N_SAMPLES_TO_USE, random_state=42)
#     else:
#         df = df_full

#     # --- The rest of your script now runs on the smaller 'df' DataFrame ---

#     with open(SYMPTOM_MAP_PATH, 'r', encoding='utf-8') as f:
#         symptom_map_json = json.load(f)
#     symptom_map = {k: v['name'].replace('_', ' ') for k, v in symptom_map_json.items()
#                    if isinstance(v, dict) and 'name' in v}

#     # --- 2. Preprocess Data ---
#     print("\n[2/9] Preprocessing data...")
#     df['AGE'] = df['AGE'].fillna(df['AGE'].median())
#     df['EVIDENCES_LIST'] = df['EVIDENCES'].apply(ast.literal_eval)
#     df['symptom_text'] = df['EVIDENCES_LIST'].apply(lambda codes: " ".join([symptom_map.get(code, "") for code in codes]))

#     # --- 3. Generate Structured Features ---
#     print("\n[3/9] Generating structured features...")
#     mlb = MultiLabelBinarizer()
#     symptom_features = mlb.fit_transform(df['EVIDENCES_LIST'])
#     df_symptoms = pd.DataFrame(symptom_features, columns=mlb.classes_, index=df.index)

#     # --- 4. Generate Contextual Features ---
#     print("\n[4/9] Generating contextual features...")
#     tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
#     text_features_tfidf = tfidf.fit_transform(df['symptom_text'])
#     svd = TruncatedSVD(n_components=100, random_state=42)
#     contextual_features = svd.fit_transform(text_features_tfidf)
#     df_context = pd.DataFrame(contextual_features, index=df.index, columns=[f'context_{i}' for i in range(100)])

#     # --- 5. Create Target Variable ---
#     print("\n[5/9] Creating target variable...")
#     high_risk = ['Myocardial infarction', 'Pneumonia', 'Appendicitis', 'Pulmonary embolism']
#     moderate_risk = ['Malaria', 'Dengue', 'Tuberculosis', 'Gastroenteritis']
#     df['risk_level'] = df['PATHOLOGY'].apply(lambda d: 'High Risk' if d in high_risk else 'Moderate Risk' if d in moderate_risk else 'Low Risk')

#     # --- 6. Combine Features & Augment Data ---
#     print("\n[6/9] Combining features...")
#     sex_encoded = pd.get_dummies(df['SEX'], drop_first=True, prefix='SEX')
#     if 'SEX_M' not in sex_encoded.columns: sex_encoded['SEX_M'] = 0

#     df_processed = pd.concat([df[['AGE', 'risk_level']], sex_encoded[['SEX_M']], df_symptoms, df_context], axis=1).fillna(0)

#     # EDIT: Augment ONLY the binary symptom features
#     df_augmented = augment_data(df_processed, df_symptoms.columns.tolist())

#     # --- 7. Finalize Data for Training ---
#     X = df_augmented.drop('risk_level', axis=1)
#     y_raw = df_augmented['risk_level']

#     # EDIT: Create and fit the LabelEncoder ONCE
#     le = LabelEncoder().fit(y_raw)
#     y = le.transform(y_raw)

#     X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

#     # --- 8. Train Model ---
#     print("\n[8/9] Training model...")
#     sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)

#     model = xgb.XGBClassifier(
#         objective='multi:softmax', eval_metric='mlogloss', n_estimators=300, max_depth=6,early_stopping_rounds=20,
#         learning_rate=0.1, subsample=0.8, colsample_bytree=0.8,
#         reg_alpha=0.1, reg_lambda=1, random_state=42,
#         tree_method='hist', device='cuda'
#     )

#     model.fit(X_train, y_train, sample_weight=sample_weights,
#               eval_set=[(X_test, y_test)], verbose=False)

#     # --- 9. Evaluate & Save Artifacts ---
#     print("\n[9/9] Evaluating and saving artifacts...")
#     y_pred = model.predict(X_test)
#     print("\nModel Evaluation Report:")
#     print(classification_report(y_test, y_pred, target_names=le.classes_))

#     artifacts = {
#         'model': model, 'label_encoder': le, 'symptom_encoder': mlb,
#         'tfidf': tfidf, 'svd': svd, 'symptom_map': symptom_map,
#         'feature_columns': X.columns.tolist()
#     }

#     for name, artifact in artifacts.items():
#         joblib.dump(artifact, os.path.join(OUTPUT_DIR, f'{name}.pkl'))

#     print(f"\n✅ All artifacts saved to {OUTPUT_DIR}")
#     print("✅ Training pipeline completed successfully!")

In [4]:
# # predict.py
# #tf-idf
# import pandas as pd
# import numpy as np
# import ast
# import joblib
# import os
# import json
# from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
# from sklearn.feature_extraction.text import TfidfVectorizer
# from sklearn.decomposition import TruncatedSVD

# # --- CONFIGURATION ---
# TEST_PATH = '/content/drive/MyDrive/Medaid/demo.csv'
# ARTIFACTS_DIR = 'model_outputs/'
# OUTPUT_FILE = os.path.join(ARTIFACTS_DIR, 'predictions.csv')

# # --- MAIN PREDICTION SCRIPT ---
# if __name__ == "__main__":
#     print("=== MEDICAL RISK PREDICTION ===")

#     # 1. Load all saved artifacts
#     print("\n[1/4] Loading trained artifacts...")
#     try:
#         model = joblib.load(os.path.join(ARTIFACTS_DIR, 'model.pkl'))
#         le = joblib.load(os.path.join(ARTIFACTS_DIR, 'label_encoder.pkl'))
#         mlb = joblib.load(os.path.join(ARTIFACTS_DIR, 'symptom_encoder.pkl'))
#         tfidf = joblib.load(os.path.join(ARTIFACTS_DIR, 'tfidf.pkl'))
#         svd = joblib.load(os.path.join(ARTIFACTS_DIR, 'svd.pkl'))
#         feature_columns = joblib.load(os.path.join(ARTIFACTS_DIR, 'feature_columns.pkl'))
#         symptom_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'symptom_map.pkl'))
#         print("✅ All artifacts loaded successfully.")
#     except FileNotFoundError as e:
#         print(f"❌ Error loading artifacts: {e}. Please run the training script first.")
#         exit()

#     # 2. Load and preprocess new data
#     print("\n[2/4] Loading and preprocessing new data...")
#     df_new = pd.read_csv(TEST_PATH)

#     # Apply the exact same preprocessing steps using the loaded artifacts
#     df_new['AGE'] = df_new['AGE'].fillna(df_new['AGE'].median())
#     df_new['EVIDENCES_LIST'] = df_new['EVIDENCES'].apply(ast.literal_eval)
#     df_new['symptom_text'] = df_new['EVIDENCES_LIST'].apply(lambda codes: " ".join([symptom_map.get(code, "") for code in codes]))

#     symptom_features = mlb.transform(df_new['EVIDENCES_LIST'])
#     df_symptoms = pd.DataFrame(symptom_features, columns=mlb.classes_, index=df_new.index)

#     text_features_tfidf = tfidf.transform(df_new['symptom_text'])
#     contextual_features = svd.transform(text_features_tfidf)
#     df_context = pd.DataFrame(contextual_features, index=df_new.index, columns=[f'context_{i}' for i in range(100)])

#     sex_encoded = pd.get_dummies(df_new['SEX'], drop_first=True, prefix='SEX')
#     if 'SEX_M' not in sex_encoded.columns: sex_encoded['SEX_M'] = 0

#     X_new = pd.concat([df_new[['AGE']], sex_encoded[['SEX_M']], df_symptoms, df_context], axis=1)

#     # 3. Align features with training schema (CRITICAL FIX)
#     print("\n[3/4] Aligning features with training schema...")
#     # Create a blueprint with the original training columns
#     aligned_df = pd.DataFrame(columns=feature_columns)
#     # Add the new data, which will create NaNs for misaligned columns
#     aligned_df = pd.concat([aligned_df, X_new], axis=0, sort=False)
#     # Fill missing columns with 0 and drop extra columns
#     aligned_df = aligned_df.fillna(0)[feature_columns]

#     # 4. Make predictions
#     print("\n[4/4] Making predictions...")
#     predictions_encoded = model.predict(aligned_df)
#     risk_levels = le.inverse_transform(predictions_encoded)

#     results = pd.DataFrame({'PREDICTED_RISK': risk_levels})
#     results.to_csv(OUTPUT_FILE, index=False)
#     print(f"✅ Predictions saved to {OUTPUT_FILE}")

#     print("\n=== PREDICTION RESULTS ===")
#     print(results.head())

In [5]:
pip install sentence-transformers xgboost

Note: you may need to restart the kernel to use updated packages.


In [6]:
pip install faiss-cpu


Note: you may need to restart the kernel to use updated packages.


In [7]:
# import os

# file_path = r"F:\shivani\VSCode\projects\Medaid\disease_dataset\release_train_patients\release_train_patients"
# print(os.path.getsize(file_path), "bytes")

# with open(file_path, "rb") as f:
#     print(f.read(200))  # read first 200 bytes




In [4]:
# train.py
import pandas as pd
import numpy as np
import ast
import joblib
import os
import json
import faiss
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# --- CONFIGURATION ---
TRAIN_PATH = r"F:\shivani\VSCode\projects\Medaid\disease_dataset\release_train_patients\release_train_patients.csv"
SYMPTOM_MAP_PATH = r"F:\shivani\VSCode\projects\Medaid\disease_dataset\release_evidences.json"
OUTPUT_DIR = 'model_outputs/'
os.makedirs(OUTPUT_DIR, exist_ok=True)
N_SAMPLES_TO_USE = 100000

# --- MAIN SCRIPT ---
if __name__ == "__main__":
    print("=== MEDICAL RISK PREDICTION - TRAINING PHASE ===")
    
    # --- 1. Load and Downsample Data ---
    print(f"\n[1/6] Loading and downsampling data to {N_SAMPLES_TO_USE} samples...")
    df_full = pd.read_csv(TRAIN_PATH)
    df = df_full.sample(n=N_SAMPLES_TO_USE, random_state=42) if len(df_full) > N_SAMPLES_TO_USE else df_full
    
    with open(SYMPTOM_MAP_PATH, 'r', encoding='utf-8') as f:
        symptom_map_json = json.load(f)
    symptom_map = {k: v['name'].replace('_', ' ') for k, v in symptom_map_json.items() if isinstance(v, dict) and 'name' in v}
    print("✅ Data loaded.")

    # --- 2. Preprocess Data ---
    print("\n[2/6] Preprocessing data...")
    df['AGE'] = df['AGE'].fillna(df['AGE'].median())
    df['EVIDENCES_LIST'] = df['EVIDENCES'].apply(ast.literal_eval)
    df['symptom_text'] = df['EVIDENCES_LIST'].apply(lambda codes: " ".join([symptom_map.get(code, "") for code in codes]))

    # --- 3. Generate All Features ---
    print("\n[3/6] Generating features...")
    mlb = MultiLabelBinarizer()
    symptom_features = mlb.fit_transform(df['EVIDENCES_LIST'])
    df_symptoms = pd.DataFrame(symptom_features, columns=mlb.classes_, index=df.index)
    
    tfidf = TfidfVectorizer(max_features=5000, stop_words='english')
    text_features_tfidf = tfidf.fit_transform(df['symptom_text'])
    
    svd = TruncatedSVD(n_components=100, random_state=42)
    contextual_features = svd.fit_transform(text_features_tfidf)
    df_context = pd.DataFrame(contextual_features, index=df.index, columns=[f'context_{i}' for i in range(100)])
    
    # --- 4. Create Final Feature Library & Labels ---
    print("\n[4/6] Combining features and creating BALANCED risk categories...")
    sex_encoded = pd.get_dummies(df['SEX'], drop_first=True, prefix='SEX')
    X_library = pd.concat([df['AGE'], sex_encoded, df_symptoms, df_context], axis=1).fillna(0)
    
    high_risk = [
        'Myocardial infarction', 'Pulmonary embolism', 'Pneumonia', 'Appendicitis',
        'Anaphylaxis', 'Unstable angina', 'Panic attack', 'Acute rhinosinusitis',
        'Bronchitis', 'Possible NSTEMI', 'Scombroid food poisoning', 
        'Acute dystonic reaction', 'Boerhaave syndrome', 'Chagas disease',
    ]
    
    moderate_risk = [
        'Malaria', 'Dengue', 'Tuberculosis', 'Gastroenteritis', 'Anemia',
        'HIV (initial infection)', 'GERD', 'Localized edema', 'Cluster headache',
        'Atrial fibrillation', 'Acute laryngitis', 'Spontaneous pneumothorax',
        'Guillain-Barré syndrome', 'Myasthenia gravis', 'Influenza', 
        'Bronchospasm / acute asthma exacerbation', 'Acute pulmonary edema',
        'Pericarditis', 'Spontaneous rib fracture',
    ]
    
    # Assign risk levels based on pathology. This will be the label for risk prediction.
    y_risk_levels_raw = df['PATHOLOGY'].apply(
        lambda pathology: 'High Risk' if pathology in high_risk 
        else 'Moderate Risk' if pathology in moderate_risk 
        else 'Low Risk'
    )

    # Encode risk levels
    le = LabelEncoder().fit(y_risk_levels_raw)
    y_library_encoded = le.transform(y_risk_levels_raw)

    # --- 5. Build and Save Faiss Index ---
    print("\n[5/6] Building the Faiss similarity search index...")
    X_library_np = np.ascontiguousarray(X_library.values.astype('float32'))
    faiss.normalize_L2(X_library_np)
    index = faiss.IndexFlatIP(X_library_np.shape[1])
    index.add(X_library_np)
    faiss.write_index(index, os.path.join(OUTPUT_DIR, 'patient_library.index'))
    print("✅ FAISS index built and saved.")

    # --- 6. Save All Other Artifacts ---
    print("\n[6/6] Saving all other artifacts...")
    np.save(os.path.join(OUTPUT_DIR, 'patient_library_labels.npy'), y_library_encoded)
    np.save(os.path.join(OUTPUT_DIR, 'patient_library_pathologies.npy'), df['PATHOLOGY'].values)
    joblib.dump(le, os.path.join(OUTPUT_DIR, 'risk_label_encoder.pkl'))
    joblib.dump(mlb, os.path.join(OUTPUT_DIR, 'symptom_encoder.pkl'))
    joblib.dump(tfidf, os.path.join(OUTPUT_DIR, 'tfidf_model.pkl'))
    joblib.dump(svd, os.path.join(OUTPUT_DIR, 'svd_model.pkl'))
    joblib.dump(symptom_map, os.path.join(OUTPUT_DIR, 'symptom_map.pkl'))
    joblib.dump(list(X_library.columns), os.path.join(OUTPUT_DIR, 'final_feature_columns.pkl'))

    # Generate and save a simple layman's map for pathologies
    unique_pathologies = df['PATHOLOGY'].unique()
    layman_map = {p: p.replace('_', ' ').title() for p in unique_pathologies}
    joblib.dump(layman_map, os.path.join(OUTPUT_DIR, 'layman_map.pkl'))
    
    print(f"\n✅ All artifacts saved to '{OUTPUT_DIR}' directory.")


=== MEDICAL RISK PREDICTION - TRAINING PHASE ===

[1/6] Loading and downsampling data to 100000 samples...
✅ Data loaded.

[2/6] Preprocessing data...

[3/6] Generating features...

[4/6] Combining features and creating BALANCED risk categories...

[5/6] Building the Faiss similarity search index...
✅ FAISS index built and saved.

[6/6] Saving all other artifacts...

✅ All artifacts saved to 'model_outputs/' directory.


In [5]:
# import numpy as np
# import joblib
# import os

# ARTIFACTS_DIR = 'model_outputs/'

# def check_artifacts():
#     print("=== ARTIFACTS DIAGNOSTIC CHECK ===")
    
#     # Check what files exist
#     print("\nFiles in artifacts directory:")
#     if os.path.exists(ARTIFACTS_DIR):
#         for file in os.listdir(ARTIFACTS_DIR):
#             file_path = os.path.join(ARTIFACTS_DIR, file)
#             size_mb = os.path.getsize(file_path) / (1024 * 1024)
#             print(f"  {file}: {size_mb:.1f} MB")
#     else:
#         print(f"  Directory {ARTIFACTS_DIR} not found!")
#         return
    
#     # Try to load each numpy file
#     numpy_files = ['patient_library_vectors.npy', 'patient_library_labels.npy']
    
#     for npy_file in numpy_files:
#         filepath = os.path.join(ARTIFACTS_DIR, npy_file)
#         if os.path.exists(filepath):
#             print(f"\n--- Checking {npy_file} ---")
#             try:
#                 # First try without allow_pickle
#                 data = np.load(filepath)
#                 print(f"  ✅ Loaded successfully (no pickle needed)")
#                 print(f"  Shape: {data.shape}")
#                 print(f"  Data type: {data.dtype}")
#                 print(f"  Sample values: {data.flat[:5]}")
#             except ValueError as e:
#                 if "allow_pickle=False" in str(e):
#                     print(f"  ⚠️  Requires pickle, trying with allow_pickle=True...")
#                     try:
#                         data = np.load(filepath, allow_pickle=True)
#                         print(f"  ✅ Loaded with pickle")
#                         print(f"  Shape: {data.shape}")
#                         print(f"  Data type: {data.dtype}")
#                         if data.dtype == 'object':
#                             print(f"  Sample values: {list(data.flat[:3])}")
#                         else:
#                             print(f"  Sample values: {data.flat[:5]}")
#                     except Exception as e2:
#                         print(f"  ❌ Failed even with pickle: {e2}")
#                 else:
#                     print(f"  ❌ Other error: {e}")
#             except Exception as e:
#                 print(f"  ❌ Unexpected error: {e}")
#         else:
#             print(f"\n--- {npy_file} NOT FOUND ---")
    
#     # Check pickle files
#     pickle_files = ['symptom_encoder.pkl', 'tfidf_model.pkl', 'svd_model.pkl', 
#                    'symptom_map.pkl', 'final_feature_columns.pkl']
    
#     print(f"\n--- Checking pickle files ---")
#     for pkl_file in pickle_files:
#         filepath = os.path.join(ARTIFACTS_DIR, pkl_file)
#         if os.path.exists(filepath):
#             try:
#                 data = joblib.load(filepath)
#                 print(f"  ✅ {pkl_file}: loaded successfully")
#                 if pkl_file == 'final_feature_columns.pkl':
#                     print(f"      Number of features: {len(data)}")
#                 elif pkl_file == 'symptom_map.pkl':
#                     print(f"      Number of symptoms: {len(data)}")
#             except Exception as e:
#                 print(f"  ❌ {pkl_file}: {e}")
#         else:
#             print(f"  ❌ {pkl_file}: NOT FOUND")
    
#     print(f"\n=== DIAGNOSIS COMPLETE ===")

# if __name__ == "__main__":
#     check_artifacts())
#             size_mb = os.path.getsize(file_path) / (1024 * 1024)
#             print(f"  {file}: {size_mb:.1f} MB")
#     else:
#         print(f"  Directory {ARTIFACTS_DIR} not found!")
#         return
    
#     # Try to load each numpy file
#     numpy_files = ['patient_library_vectors.npy', 'patient_library_labels.npy']
    
#     for npy_file in numpy_files:
#         filepath = os.path.join(ARTIFACTS_DIR, npy_file)
#         if os.path.exists(filepath):
#             print(f"\n--- Checking {npy_file} ---")
#             try:
#                 # First try without allow_pickle
#                 data = np.load(filepath)
#                 print(f"  ✅ Loaded successfully (no pickle needed)")
#                 print(f"  Shape: {data.shape}")
#                 print(f"  Data type: {data.dtype}")
#                 print(f"  Sample values: {data.flat[:5]}")
#             except ValueError as e:
#                 if "allow_pickle=False" in str(e):
#                     print(f"  ⚠️  Requires pickle, trying with allow_pickle=True...")
#                     try:
#                         data = np.load(filepath, allow_pickle=True)
#                         print(f"  ✅ Loaded with pickle")
#                         print(f"  Shape: {data.shape}")
#                         print(f"  Data type: {data.dtype}")
#                         if data.dtype == 'object':
#                             print(f"  Sample values: {list(data.flat[:3])}")
#                         else:
#                             print(f"  Sample values: {data.flat[:5]}")
#                     except Exception as e2:
#                         print(f"  ❌ Failed even with pickle: {e2}")
#                 else:
#                     print(f"  ❌ Other error: {e}")
#             except Exception as e:
#                 print(f"  ❌ Unexpected error: {e}")
#         else:
#             print(f"\n--- {npy_file} NOT FOUND ---")
    
#     # Check pickle files
#     pickle_files = ['symptom_encoder.pkl', 'tfidf_model.pkl', 'svd_model.pkl', 
#                    'symptom_map.pkl', 'final_feature_columns.pkl']
    
#     print(f"\n--- Checking pickle files ---")
#     for pkl_file in pickle_files:
#         filepath = os.path.join(ARTIFACTS_DIR, pkl_file)
#         if os.path.exists(filepath):
#             try:
#                 data = joblib.load(filepath)
#                 print(f"  ✅ {pkl_file}: loaded successfully")
#                 if pkl_file == 'final_feature_columns.pkl':
#                     print(f"      Number of features: {len(data)}")
#                 elif pkl_file == 'symptom_map.pkl':
#                     print(f"      Number of symptoms: {len(data)}")
#             except Exception as e:
#                 print(f"  ❌ {pkl_file}: {e}")
#         else:
#             print(f"  ❌ {pkl_file}: NOT FOUND")
    
#     print(f"\n=== DIAGNOSIS COMPLETE ===")

# if __name__ == "__main__":
#     check_artifacts()

In [6]:
# predict.py
import pandas as pd
import numpy as np
import ast
import joblib
import os
import faiss
from collections import Counter
from sklearn.preprocessing import MultiLabelBinarizer, LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD

# --- CONFIGURATION ---
TEST_PATH = r"F:\shivani\VSCode\projects\Medaid\disease_dataset\release_test_patients\release_test_patients" # Your messy, manual test file
ARTIFACTS_DIR = 'model_outputs/'
OUTPUT_FILE = os.path.join(ARTIFACTS_DIR, 'predictions.csv')
K_NEIGHBORS = 15 # Number of similar cases to consider for voting
RISK_WEIGHTS = {
    'High Risk': 1.5,   # Boost high-risk cases in voting
    'Moderate Risk': 1.2, # Slight boost for moderate risk
    'Low Risk': 1.0       # Standard weight
}

# --- MAIN PREDICTION SCRIPT ---
if __name__ == "__main__":
    print("=== MEDICAL RISK PREDICTION (Similarity Search) ===")
    
    # 1. Load the Case Library and all artifacts
    print("\n[1/4] Loading case library and artifacts...")
    try:
        index = faiss.read_index(os.path.join(ARTIFACTS_DIR, 'patient_library.index'))
        y_library_encoded = np.load(os.path.join(ARTIFACTS_DIR, 'patient_library_labels.npy'), allow_pickle=True)
        y_pathologies = np.load(os.path.join(ARTIFACTS_DIR, 'patient_library_pathologies.npy'), allow_pickle=True)
        le = joblib.load(os.path.join(ARTIFACTS_DIR, 'risk_label_encoder.pkl'))
        mlb = joblib.load(os.path.join(ARTIFACTS_DIR, 'symptom_encoder.pkl'))
        tfidf = joblib.load(os.path.join(ARTIFACTS_DIR, 'tfidf_model.pkl'))
        svd = joblib.load(os.path.join(ARTIFACTS_DIR, 'svd_model.pkl'))
        feature_columns = joblib.load(os.path.join(ARTIFACTS_DIR, 'final_feature_columns.pkl'))
        symptom_map = joblib.load(os.path.join(ARTIFACTS_DIR, 'symptom_map.pkl'))
        risk_labels_map = {i: label for i, label in enumerate(le.classes_)}
        print("✅ Artifacts loaded.")
    except FileNotFoundError as e:
        print(f"❌ Error loading artifacts: {e}. Please run the train.py script first.")
        exit()

    # 2. Load and preprocess the new, unseen data
    print(f"\n[2/4] Loading and preprocessing new data from '{TEST_PATH}'...")
    df_new = pd.read_csv(TEST_PATH)
    df_new['AGE'] = df_new['AGE'].fillna(df_new['AGE'].median())
    df_new['EVIDENCES_LIST'] = df_new['EVIDENCES'].apply(ast.literal_eval)
    df_new['symptom_text'] = df_new['EVIDENCES_LIST'].apply(lambda codes: " ".join([symptom_map.get(code, "") for code in codes]))
    symptom_features = mlb.transform(df_new['EVIDENCES_LIST'])
    df_symptoms = pd.DataFrame(symptom_features, columns=mlb.classes_)
    text_features_tfidf = tfidf.transform(df_new['symptom_text'])
    contextual_features = svd.transform(text_features_tfidf)
    df_context = pd.DataFrame(contextual_features, columns=[f'context_{i}' for i in range(100)])
    sex_encoded = pd.get_dummies(df_new['SEX'], drop_first=True, prefix='SEX')
    if 'SEX_M' not in sex_encoded.columns: sex_encoded['SEX_M'] = 0
    X_new_unaligned = pd.concat([df_new['AGE'], sex_encoded[['SEX_M']], df_symptoms, df_context], axis=1)
    X_new_aligned = X_new_unaligned.reindex(columns=feature_columns, fill_value=0)

    # 3. Find Similar Cases with Faiss
    print(f"\n[3/4] Searching for {K_NEIGHBORS} most similar cases using Faiss...")
    X_new_np = np.ascontiguousarray(X_new_aligned.values.astype('float32'))
    faiss.normalize_L2(X_new_np)
    
    distances, top_k_indices = index.search(X_new_np, K_NEIGHBORS)
    
    # 4. Predict using Weighted Voting
    print("\n[4/4] Predicting risk based on weighted voting...")
    final_predictions = []
    
    for i in range(len(top_k_indices)):
        top_labels_encoded = y_library_encoded[top_k_indices[i]]  # Fixed variable name
        top_scores = distances[i]
        
        vote_counts = Counter()
        for j, label_encoded in enumerate(top_labels_encoded):  # Fixed variable name
            label_text = risk_labels_map[label_encoded]
            risk_weight = RISK_WEIGHTS.get(label_text, 1.0)
            vote_counts[label_text] += top_scores[j] * risk_weight
        
        prediction = vote_counts.most_common(1)[0][0]
        final_predictions.append(prediction)

    # Decode predictions and save
    results_df = pd.DataFrame({'PREDICTED_RISK': final_predictions})
    results_df.to_csv(OUTPUT_FILE, index=False)
    
    print(f"\n✅ Predictions saved to {OUTPUT_FILE}")
    print("\n=== PREDICTION RESULTS ===")
    print(results_df)

=== MEDICAL RISK PREDICTION (Similarity Search) ===

[1/4] Loading case library and artifacts...
✅ Artifacts loaded.

[2/4] Loading and preprocessing new data from 'F:\shivani\VSCode\projects\Medaid\disease_dataset\release_test_patients\release_test_patients'...


C:\Users\shiva\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\preprocessing\_label.py:909: UserWarning: unknown class(es) ['E_133_@_V_162', 'E_133_@_V_61', 'E_55_@_V_152', 'E_55_@_V_24', 'E_55_@_V_45'] will be ignored
  warnings.warn(



[3/4] Searching for 15 most similar cases using Faiss...

[4/4] Predicting risk based on weighted voting...

✅ Predictions saved to model_outputs/predictions.csv

=== PREDICTION RESULTS ===
       PREDICTED_RISK
0       Moderate Risk
1           High Risk
2            Low Risk
3       Moderate Risk
4            Low Risk
...               ...
134524       Low Risk
134525       Low Risk
134526       Low Risk
134527      High Risk
134528       Low Risk

[134529 rows x 1 columns]
